In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import localtime, strftime
from collections import defaultdict

%matplotlib inline
pd.set_option('display.max_rows', 200)

In [2]:
def read_orig_file(data_path=None, orig_file=None):
    """Read the original data file into a pandas DataFrame.
    
    Parameters
    ----------
    data_path : string, optional
        directory containing original file
    orig_file : string, optional
        filename containing original file 
    
    Returns
    -------
    raw_data : DataFrame

    """
    orig_file_defaults = {
        'data_path': '/Users/panpancheng/Documents/study/capstone/dataset/',
        'orig_file': 'NYPD_Complaint_Data_Historic.csv',
        'dtype': {
            'CMPLNT_NUM': np.int64,
            'CMPLNT_FR_DT': str,
            'CMPLNT_FR_TM': str,
            'RPT_DT': str,
            'KY_CD': np.int32,
            'OFNS_DESC': str,
            'LAW_CAT_CD': str,
            'BORO_NM': str,
            'ADDR_PCT_CD': str,
            'Latitude': np.float64,
            'Longitude': np.float64
        },
        'index_col': 'CMPLNT_NUM',
        'usecols': [
            'CMPLNT_NUM',
            'CMPLNT_FR_DT',
            'CMPLNT_FR_TM',
            'RPT_DT',
            'KY_CD',
            'OFNS_DESC',
            'LAW_CAT_CD',
            'BORO_NM',
            'ADDR_PCT_CD',
            'Latitude',
            'Longitude'
        ],
        'parse_dates_cols': ['RPT_DT'],
    }
    
    if data_path is None:
        data_path = orig_file_defaults['data_path']
    if orig_file is None:
        orig_file = orig_file_defaults['orig_file']
        
    raw_data = pd.read_csv(
        filepath_or_buffer = data_path+orig_file,
        index_col = orig_file_defaults['index_col'],
        usecols = orig_file_defaults['usecols'],
        dtype = orig_file_defaults['dtype'],
        parse_dates = orig_file_defaults['parse_dates_cols'],
        infer_datetime_format = True,
    )
    return raw_data

In [3]:
def filter_raw_data(raw_data, output_file=None):
    """Get rid of useless rows.

    Removes non-felonies or rows with nonexistent report dates.
    
    Parameters
    ----------
    raw_data : DataFrame
    output_file : string
    
    Returns
    -------
    nypd_data : DataFrame

    """

    if output_file is None:
        output_file = '/Users/panpancheng/Documents/study/capstone/dataset/raw_dated_felonies.csv'

    raw_data.dropna(
        subset=['CMPLNT_FR_DT', 'CMPLNT_FR_TM']
    )
    raw_data = raw_data[raw_data['LAW_CAT_CD'] == 'FELONY']
    raw_data = raw_data[pd.to_numeric(raw_data['ADDR_PCT_CD'], errors='coerce').fillna(-1) != -1]
    raw_data.to_csv(output_file)

In [4]:
def save_dated_felonies(output_file=None):
    print('Starting ({0})...'.format(strftime("%Y-%m-%d %H:%M:%S", localtime())))
    raw_data = read_orig_file()
    print('Saving filtered output ({0})...'.format(strftime("%Y-%m-%d %H:%M:%S", localtime())))
    filter_raw_data(raw_data, output_file)
    print('Done ({0})'.format(strftime("%Y-%m-%d %H:%M:%S", localtime())))

In [5]:
def load_dated_felonies(data_path=None, filtered_file=None):
    filtered_file_defaults = {
        'data_path': '/Users/panpancheng/Documents/study/capstone/dataset/',
        'filtered_file': 'raw_dated_felonies.csv',
        'dtype': {
            'CMPLNT_NUM': np.int64,
            'CMPLNT_FR_DT': str,
            'CMPLNT_FR_TM': str,
            'RPT_DT': str,
            'KY_CD': np.int32,
            'OFNS_DESC': str,
            'BORO_NM': str,
            'ADDR_PCT_CD': np.int32,
            'Latitude': np.float64,
            'Longitude': np.float64,
        },
        'index_col': 'CMPLNT_NUM',
        'usecols': [
            'CMPLNT_NUM',
            'CMPLNT_FR_DT',
            'CMPLNT_FR_TM',
            'RPT_DT',
            'KY_CD',
            'OFNS_DESC',
            'BORO_NM',
            'ADDR_PCT_CD',
            'Latitude',
            'Longitude',
        ],
        'parse_dates_dict': {
            'COMPLAINT_DATETIME': ['CMPLNT_FR_DT', 'CMPLNT_FR_TM'],
            'REPORT_DATE': ['RPT_DT'],
        },
    }
    
    if data_path is None:
        data_path = filtered_file_defaults['data_path']
    if filtered_file is None:
        filtered_file = filtered_file_defaults['filtered_file']
        
    nypd_data = pd.read_csv(
        filepath_or_buffer = data_path+filtered_file,
        index_col = filtered_file_defaults['index_col'],
        usecols = filtered_file_defaults['usecols'],
        dtype = filtered_file_defaults['dtype'],
        parse_dates = filtered_file_defaults['parse_dates_dict'],
        infer_datetime_format = True,
    )
    nypd_data['COMPLAINT_DATETIME'] = pd.to_datetime(nypd_data['COMPLAINT_DATETIME'], errors='coerce')
    nypd_data.dropna(subset=['COMPLAINT_DATETIME'])
    return nypd_data[nypd_data['COMPLAINT_DATETIME'] >= '2006-01-02 00:00:00'] # Weird data on 2006-01-01.

In [6]:
def save_clean_felonies(output_file=None):
    if output_file is None:
        output_file = '/Users/panpancheng/Documents/study/capstone/dataset/clean_felonies.csv'
    print('Starting ({0})...'.format(strftime("%Y-%m-%d %H:%M:%S", localtime())))
    filtered_felonies = load_dated_felonies()
    print('Done ({0})'.format(strftime("%Y-%m-%d %H:%M:%S", localtime())))
    filtered_felonies.to_csv(output_file)

In [7]:
def load_clean_felonies(data_path=None, clean_file=None):
    clean_file_defaults = {
        'data_path': '/Users/panpancheng/Documents/study/capstone/dataset/',
        'clean_file': 'clean_felonies.csv',
        'dtype': {
            'CMPLNT_NUM': np.int64,
            'COMPLAINT_DATETIME': str,
            'REPORT_DATE': str,
            'KY_CD': np.int32,
            'OFNS_DESC': str,
            'BORO_NM': str,
            'ADDR_PCT_CD': np.int32,
            'Latitude': np.float64,
            'Longitude': np.float64,
        },
        'index_col': 'CMPLNT_NUM',
        'usecols': [
            'CMPLNT_NUM',
            'COMPLAINT_DATETIME',
            'REPORT_DATE',
            'KY_CD',
            'OFNS_DESC',
            'BORO_NM',
            'ADDR_PCT_CD',
            'Latitude',
            'Longitude',
        ],
        'parse_dates_cols': ['REPORT_DATE', 'COMPLAINT_DATETIME'],
    }
    
    if data_path is None:
        data_path = clean_file_defaults['data_path']
    if clean_file is None:
        clean_file = clean_file_defaults['clean_file']
        
    nypd_data = pd.read_csv(
        filepath_or_buffer = data_path+clean_file,
        index_col = clean_file_defaults['index_col'],
        usecols = clean_file_defaults['usecols'],
        dtype = clean_file_defaults['dtype'],
        parse_dates = clean_file_defaults['parse_dates_cols'],
        infer_datetime_format = True,
    )
    
    nypd_data.sort_values(by='COMPLAINT_DATETIME', inplace=True)
    return nypd_data


In [8]:
def add_offense_category(df):
    offense_category = defaultdict(lambda : 'Other')

    offense_category[101] = 'Homicide'
    offense_category[102] = 'Homicide'
    offense_category[103] = 'Homicide'

    offense_category[104] = 'Rape'
    offense_category[116] = 'Rape'
    
    offense_category[105] = 'Robbery'           # Mugging
    offense_category[106] = 'FelonyAssault'
    offense_category[107] = 'Burglary'          # Breaking and entering
    offense_category[109] = 'GrandLarceny'      
    offense_category[110] = 'GrandLarcenyAuto'
    
    offense_category[112] = 'Fraud'
    offense_category[113] = 'Forgery'
    offense_category[114] = 'Arson'
    offense_category[117] = 'Drugs'
    offense_category[118] = 'Weapons'
    offense_category[121] = 'CriminalMischief'  # Graffiti
    
    df['OFFENSE'] = df['KY_CD'].map(offense_category).astype('category')
    df['OFFENSE'].cat.set_categories([
        'Homicide', 'Rape', 'Robbery', 'FelonyAssault', 'Burglary', 'GrandLarceny', 'GrandLarcenyAuto',
        'Fraud', 'Forgery', 'Arson', 'Drugs', 'Weapons', 'CriminalMischief', 'Other'
    ], inplace=True)

In [9]:
# Before running: execute "tar -xvf cf.tar.gz" from the root directory of the git repo
# copy clean_felonies.csv into that directory,
# and unzip it 
#
#
# To make the files from scratch (not necessary):
save_dated_felonies()
save_clean_felonies()


nypd_data = load_clean_felonies()
add_offense_category(nypd_data)

Starting (2017-10-28 16:01:28)...


/Users/panpancheng/.pyenv/versions/3.6.2/envs/vir3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Saving filtered output (2017-10-28 16:02:02)...
Done (2017-10-28 16:02:29)
Starting (2017-10-28 16:02:30)...
Done (2017-10-28 16:09:20)


In [10]:
nypd_data.head(5)

,COMPLAINT_DATETIME,REPORT_DATE,KY_CD,OFNS_DESC,BORO_NM,ADDR_PCT_CD,Latitude,Longitude,OFFENSE
CMPLNT_NUM,,,,,,,,,
318229393,2006-01-02 00:00:00,2006-01-02,109,GRAND LARCENY,MANHATTAN,18,40.759529,-73.984397,GrandLarceny
554420424,2006-01-02 00:00:00,2008-11-26,112,THEFT-FRAUD,BROOKLYN,68,40.630222,-74.023848,Fraud
218039893,2006-01-02 00:00:00,2006-11-26,109,GRAND LARCENY,MANHATTAN,6,40.733916,-74.000781,GrandLarceny
542037522,2006-01-02 00:00:00,2006-01-02,126,MISCELLANEOUS PENAL LAW,BROOKLYN,76,40.675672,-74.006256,Other
492019346,2006-01-02 00:01:00,2006-01-07,109,GRAND LARCENY,BROOKLYN,70,40.634735,-73.952237,GrandLarceny


In [11]:
nypd_data.to_csv('clean_felonies_offense.csv')

In [23]:
offense_list = list(set(nypd_data['OFFENSE']))
offense_list

['Arson',
 'Weapons',
 'Fraud',
 'GrandLarceny',
 'Robbery',
 'Burglary',
 'Homicide',
 'Drugs',
 'Other',
 'Rape',
 'GrandLarcenyAuto',
 'CriminalMischief',
 'FelonyAssault',
 'Forgery']

In [22]:
def add_offense_category(df, offense_list):
    for offense in offense_list:
        df[offense] = (df['OFFENSE'] == offense).astype(int)


,COMPLAINT_DATETIME,REPORT_DATE,KY_CD,OFNS_DESC,BORO_NM,ADDR_PCT_CD,Latitude,Longitude,OFFENSE,Arson,...,Robbery,Burglary,Homicide,Drugs,Other,Rape,GrandLarcenyAuto,CriminalMischief,FelonyAssault,Forgery
CMPLNT_NUM,,,,,,,,,,,,,,,,,,,,,
318229393,2006-01-02 00:00:00,2006-01-02,109,GRAND LARCENY,MANHATTAN,18,40.759529,-73.984397,GrandLarceny,0,...,0,0,0,0,0,0,0,0,0,0
554420424,2006-01-02 00:00:00,2008-11-26,112,THEFT-FRAUD,BROOKLYN,68,40.630222,-74.023848,Fraud,0,...,0,0,0,0,0,0,0,0,0,0
218039893,2006-01-02 00:00:00,2006-11-26,109,GRAND LARCENY,MANHATTAN,6,40.733916,-74.000781,GrandLarceny,0,...,0,0,0,0,0,0,0,0,0,0
542037522,2006-01-02 00:00:00,2006-01-02,126,MISCELLANEOUS PENAL LAW,BROOKLYN,76,40.675672,-74.006256,Other,0,...,0,0,0,0,1,0,0,0,0,0
492019346,2006-01-02 00:01:00,2006-01-07,109,GRAND LARCENY,BROOKLYN,70,40.634735,-73.952237,GrandLarceny,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
nypd_data_with_category = nypd_data
add_offense_category(nypd_data_with_category, offense_list)
nypd_data_with_category.head(5)

In [24]:
nypd_data_with_category.to_csv('clean_felonies_offense.csv')

In [25]:
nypd_data_with_category.head(5)

,COMPLAINT_DATETIME,REPORT_DATE,KY_CD,OFNS_DESC,BORO_NM,ADDR_PCT_CD,Latitude,Longitude,OFFENSE,Arson,...,Robbery,Burglary,Homicide,Drugs,Other,Rape,GrandLarcenyAuto,CriminalMischief,FelonyAssault,Forgery
CMPLNT_NUM,,,,,,,,,,,,,,,,,,,,,
318229393,2006-01-02 00:00:00,2006-01-02,109,GRAND LARCENY,MANHATTAN,18,40.759529,-73.984397,GrandLarceny,0,...,0,0,0,0,0,0,0,0,0,0
554420424,2006-01-02 00:00:00,2008-11-26,112,THEFT-FRAUD,BROOKLYN,68,40.630222,-74.023848,Fraud,0,...,0,0,0,0,0,0,0,0,0,0
218039893,2006-01-02 00:00:00,2006-11-26,109,GRAND LARCENY,MANHATTAN,6,40.733916,-74.000781,GrandLarceny,0,...,0,0,0,0,0,0,0,0,0,0
542037522,2006-01-02 00:00:00,2006-01-02,126,MISCELLANEOUS PENAL LAW,BROOKLYN,76,40.675672,-74.006256,Other,0,...,0,0,0,0,1,0,0,0,0,0
492019346,2006-01-02 00:01:00,2006-01-07,109,GRAND LARCENY,BROOKLYN,70,40.634735,-73.952237,GrandLarceny,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
len(list(set(nypd_data_with_category['ADDR_PCT_CD'])))

6